In [1]:
from lyra import Client
from lyra.parser import parse_form
from lyra.transformer.transformer import transform_form
from IPython.display import display, Markdown
from lyra.translator.translate import make_collection, imports

from fundar import json
from glob import glob
from tqdm.auto import tqdm

forms = glob('../data/*-form.json')

def display_markdown(md):
    display(Markdown(md))

with open('lyra/cds_collections/cds_collections.py', 'w') as f:
    f.write(imports)
    for form in forms:
        ast = parse_form(json.load(form))
        collection_name = form.split('-form')[0].rsplit('/')[-1]

        transformed_ast = sorted(set(transform_form(ast)))
        
        #display_str.append('\n```')

        f.write(make_collection(collection_name.replace('-', '_'), transformed_ast))

In [2]:
from typing import Union

In [3]:
from lyra.cds_collections import Collection_derived_era5_land_daily_statistics

In [4]:
Collection_derived_era5_land_daily_statistics.download(

)

TypeError: Collection_derived_era5_land_daily_statistics.__download__() missing 4 required positional arguments: 'year', 'month', 'day', and 'variable'